# Classification
In this notebook, we will apply several classification algorithms on two data set: *Iris* and *Digits*. The first one is a low dimensional data set while the second one contains more features.

In [ ]:
from sklearn.datasets import load_digits, load_iris
# Load digits
X, y = load_digits(return_X_y=True) # load_iris(return_X_y=True)
print(X.shape)

# Visualization of the digits data set
The data set is made available *by NIST to extract normalized bitmaps of handwritten digits from a preprinted form. From a total of 43 people, 30 contributed to the training set and different 13 to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of 4x4 and the number of on pixels are counted in each block. This generates an input matrix of 8x8 where each element is an integer in the range 0..16* (https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits)

It is possible to visualize the data by reshaping correctly each sample:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Plot the first element
plt.figure()
plt.imshow(X[0,:].reshape(8,8),cmap="gray")
plt.title("Label = {}".format(y[0]))

# Plot the hundredth element
plt.figure()
plt.imshow(X[100,:].reshape(8,8),cmap="gray")
plt.title("Label = {}".format(y[100]))


## Classification framework
Two steps are required
- Model selection, i.e. find the optimal hyperparemeters,
- Model assessement, i.e. validate the model on unseen data.

As said in introduction, scikit-learn offers convenient and generic functions to achieve these steps. In what follow, an example is given for SVM. But it can be extended for any algorithm in scikit-learn, up to a correct definition of the hyperparameters. In this labworks, we use the support vector classifier (SVC) (http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

In [ ]:
import scipy as sp
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

# Standardize data
sc = MinMaxScaler()
X = sc.fit_transform(X) # Scale data between 0 and 1

# Split the data -> train_size=0.20 means we keep 20% of the data for training and 80% fir validation
# The stratification ensures that the proportion of each class from the orginal data is preserved in the train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.20, random_state=0, stratify=y)

With kernel methods (and more generaly for any distance based methods), it is a good practice to standardize feature remove dynamics effect. Here we rescale each feature between 0 and 1.

In [ ]:
# Definition of the hyperparameters values search
degree = sp.arange(1,7) # Degree of the polynomial kernel
C = 10.0**sp.arange(0,4) # Penality of the optimization problem
param_grid = dict(degree=degree, C=C, kernel=['poly'])

We use [cross-validation](http://scikit-learn.org/stable/modules/cross_validation.html) strategy to select the optimal set of hyperparameters.

In [ ]:
grid = GridSearchCV(SVC(),  # Set up the classifier
                    param_grid=param_grid, 
                    cv= 3,
                    n_jobs=-1) # Do the grid search in parallel
grid.fit(X_train, y_train) # Run the grid search

# Print some results
print("Best score: {}".format(grid.best_score_)) # Default scorer in scikit is the correct classification rate
print("Best set of hyperparameters: {}".format(grid.best_params_))

# Learn the optimal model
clf = grid.best_estimator_  # Get the best estimator
clf.fit(X_train,y_train)  # Fit it using the training set

# Predict new samples
y_pred = clf.predict(X_test)

# Compute the overall accuracies
print("Correct classification rate on the test set: {}".format(accuracy_score(y_test,y_pred)))

We can plot the correct classification rate function of the hyperparameters. It is important to check if our search values are correct.

In [ ]:
res = grid.cv_results_['mean_test_score'].reshape(C.size,degree.size)
X, Y = sp.meshgrid(degree, C)
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
cp = ax.contourf(X, Y, res)
ax.scatter(grid.best_params_['degree'],grid.best_params_['C'],color='k')
ax.set_yscale("log")
ax.set_xlabel("Degree")
ax.set_ylabel("C")
fig.colorbar(cp)

Now suppose we would like to compare several kernels. We need can use the same workflow, just by adding one line and modifying one line !!

In [ ]:
gamma = 2.0**sp.arange(-4,2) # Scale of the RBF kernel

In [ ]:
param_grid = [dict(kernel=['rbf'],gamma=gamma, C=C),
              dict(kernel=['poly'],degree=degree, C=C)]

We can now now copy/past/run the same code than previously:

In [ ]:
grid = GridSearchCV(SVC(),  # Set up the classifier
                    param_grid=param_grid, 
                    cv= 3,
                    n_jobs=-1) # Do the grid search in parallel

grid.fit(X_train, y_train) # Run the grid search
print("Best score: {}".format(grid.best_score_))
print("Best set of hyperparameters: {}".format(grid.best_params_))
print
# Learn the optimal model
clf = grid.best_estimator_  # Get the best estimator
clf.fit(X_train,y_train)  # Fit it using the training set

# Predict new samples
y_pred = clf.predict(X_test)

# Compute the overall accuracies
print("Correct classification rate on the test set: {}".format(accuracy_score(y_test,y_pred)))

## Comparison of the differents classifiers
In the course, we have discussed about three classifiers: QDA, SVM and K-NN. Using the following scripts, we are going to compare their performances on the given data sets. Again, using scikit-learn generic function, it is possible we few lines of code to run all the experiments.


In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
# This lines are used to remove warning from scikit-learn when QDA is used, to prevent noisy verbosity in the notebook: should be used in practice!
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Define the classifier list
classifiers = [SVC(), KNeighborsClassifier(), QuadraticDiscriminantAnalysis(), LinearDiscriminantAnalysis(solver="lsqr")]
names = ["SVM", "KNN", "QDA", "LDA"]

# Define the dictionnary of parameters to optimize
param_grids = [dict(kernel=['rbf'], gamma=gamma, C=C),
               dict(n_neighbors = sp.arange(1,40)), # number of neighbors for KNN
               dict(reg_param = sp.linspace(0,0.1,30)), # Regularization parameter for QDA
               dict(shrinkage = sp.linspace(0,0.5,30)), # Regularization parameter for LDA
               ]
 
# Run all classifiers
for classifier, name, param_grid in zip(classifiers, names, param_grids):
    grid = GridSearchCV(classifier, param_grid=param_grid, cv= 3, n_jobs=-1)
    grid.fit(X_train, y_train)
    
    clf = grid.best_estimator_ 
    clf.fit(X_train,y_train)
    
    y_pred = clf.predict(X_test)
    print("Classification accuracy for {}: {} (best parameters {})".format(name,accuracy_score(y_test,y_pred),grid.best_params_))

## Pipeline
In the previous section, we have selected the best kernel from the polynomial and RBF one. But it is also possible to combine them using [pipeline](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) facility offer by scikit-learn. The following code does the job. 

We first define the CompositeKernel class, which is the weighted summation of the RBF and polynomial kernel with their own parameters.

In [ ]:
from sklearn.metrics.pairwise import rbf_kernel, polynomial_kernel
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

# Convenient Class for summation kernel
class CompositeKernel(BaseEstimator,TransformerMixin):
    def __init__(self,
                 mu=0.5,
                 gamma=0.125,  
                 degree=3):
        self.gamma = gamma
        self.degree = degree
        self.mu = mu
        
    def transform(self,X):
        K = self.mu*rbf_kernel(X,self.X_,gamma=self.gamma)
        K += (1-self.mu)*polynomial_kernel(X,self.X_,degree=self.degree)
        return K

    def fit(self,X,y=None, **fit_params):
        self.X_ = X
        return self

Then we define the pipeline:

In [ ]:
# Create a pipeline
pipe = Pipeline([
    ('CK',CompositeKernel()),
    ('SVM',SVC())
])

Then we define the range search for the weight paramter *mu* and pass the others parameters ranges. Here we are going to optimize all the parameters using grid search, i.e., a  brut force strategy. It exists several algorithms to do it properly, e.g., https://arxiv.org/abs/1602.02355. 

In [ ]:
param_grid = dict([
    ('CK__mu', sp.linspace(0,1,num=11)),  # Note the double "_" to define parameters of a pipeline
    ('CK__gamma', gamma),
    ('CK__degree', degree),
    ('SVM__C',C),
    ('SVM__kernel', ['precomputed']),
])
grid = GridSearchCV(pipe,  # Set up the classifier -> Here we put our pipeline
                    param_grid=param_grid, 
                    cv= 3,
                    n_jobs=-1) # Do the grid search in parallel

grid.fit(X_train, y_train) # Run the grid search and WAIT: 11*6*6*4 tuples of values to be tested !
print("Best score: {}".format(grid.best_score_))
print("Best set of hyperparameters: {}".format(grid.best_params_))

# Learn the optimal model
clf = grid.best_estimator_  # Get the best estimator
clf.fit(X_train,y_train)

# Predict new samples
y_pred = clf.predict(X_test)

# Compute the overall accuracies
print(accuracy_score(y_test,y_pred))